# MAD Statique <br>
### STAGE ANCHES <br>
Camille Urban <br>
22/03/2024

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from scipy.optimize import least_squares

from scipy import ndimage
import os
import scipy.io as sio
from scipy.ndimage import rotate
from skimage.io import imread
# from skimage.transform import rotate

from im_rotate import im_rotate
# from caracNL_BG import caracNL_BG
# from identification_modele import identification_modele


In [3]:
%matplotlib
%matplotlib

Using matplotlib backend: <object object at 0x0000021E1B813A80>
Using matplotlib backend: TkAgg


## Paramètres du Banc

In [4]:
# SENSIBILITE CAPTEUR
S_force =  39.2  # mV/N
G_force = 100

# cannal d'acquisition
Cannal = 1

## Traitement données

In [5]:
# Metre le fichier . mat en .txt 

dossier = 'mes_2024.03.22/'

# surf_c = np.loadtxt(dossier + '/A1.txt')
               
# Lecture du fichier MATLAB (.m) et écriture dans un fichier texte (.txt)
with open(dossier + '/A1.mat', 'r') as file:
    matlab_code = file.read()
    
# Écriture du contenu dans un fichier texte
with open(dossier + '/A1.txt', 'w') as file:
    file.write(matlab_code)
    
print("Conversion terminée. Le fichier MATLAB a été converti en fichier texte.")

Conversion terminée. Le fichier MATLAB a été converti en fichier texte.


In [6]:
# Ouvrir le fichier d'entrée en mode lecture
with open(dossier + 'A1.txt', 'r') as f:
    lines = f.readlines()

# Initialiser une variable pour suivre le numéro de fichier en cours
current_file_index = 1

# Parcourir les lignes du fichier
for line in lines:
    if line.startswith('#'):
        # Si une ligne commence par '#', cela indique un nouveau fichier
        current_file_index += 1
    else:
        # Écrire le contenu de la ligne dans le fichier correspondant
        with open(dossier + f'/matrix{current_file_index}.txt', 'a') as current_file:
            current_file.write(line)

In [7]:
# def_r 
with open(dossier + '/matrix6.txt', 'r') as f:
    lines = f.readlines()

# Initialiser une variable pour suivre le numéro de fichier en cours
current_file_index = 1
# Parcourir les lignes du fichier
for line in lines:
    # Diviser la ligne en valeurs individuelles en utilisant l'espace comme délimiteur
    values = line.split()
    values = np.array(values)
    # Écrire les valeurs dans le fichier correspondant
    with open(dossier + '/def_r.txt', 'a') as current_file:
        for value in values:
            current_file.write(value + ' ')
        current_file.write('\n')

# der_m
with open(dossier + '/matrix10.txt', 'r') as f:
    lines = f.readlines()
current_file_index = 1
for line in lines:
    values = line.split()
    values = np.array(values)
    with open(dossier + '/def_m.txt', 'a') as current_file:
        for value in values:
            current_file.write(value + ' ')
        current_file.write('\n')
        
# surf_c
with open(dossier + '/matrix14.txt', 'r') as f:
    lines = f.readlines()
current_file_index = 1
for line in lines:
    values = line.split()
    values = np.array(values)
    with open(dossier + '/surf_c.txt', 'a') as current_file:
        for value in values:
            current_file.write(value + ' ')
        current_file.write('\n')

# Traitement images

### Extraction données image

In [22]:
plt.close('all')
### Mesures MAD Statique

# def detect_contour(dossier, nom_anche):
nom_anche = 'A'

surf_c = pd.read_table(dossier + '/surf_c.txt', sep=' ').values
def_r = pd.read_table(dossier + '/def_r.txt', sep=' ').values
def_m = pd.read_table(dossier + '/def_m.txt', sep=' ').values

# Seuil 
seuil = 1150

# on repère les bords du bec pour une seule image (le bec ne bouge pas pour les autres ensuite alors on peut garder les mêmes cibles)
J = cv2.threshold(cv2.imread(dossier + nom_anche + '1_0.bmp', cv2.IMREAD_GRAYSCALE), 128, 255, cv2.THRESH_BINARY)[1]
print(np.shape(J))
angle_rot = im_rotate(J)
angle_rot = angle_rot[0]

# repère des bords de l'anche
cibles = ['Anche Gauche', 'Anche Droite']
hmax = 350  # hauteur de l'image

for R in range (1):
    I = cv2.threshold(cv2.imread(dossier + nom_anche + str(R+1) + '_0.bmp',  cv2.IMREAD_GRAYSCALE), 128, 255, cv2.THRESH_BINARY)[1]
    I = np.flipud(I) # Reverse the order of elements along axis 0
    I = I[150:hmax, :]  # couper l'image pour ne garder qu'une partie utile

    W = I.shape[1]  # Width
    H = I.shape[0]  # Height

    X = np.arange(1,W+1) 
    Y = np.arange(1,H+1)
    a = 30
    N = 20  # taille fenêtre glissante (en px)
    dist_px = 5

    # # Trouver les bords de l'anche, on gardera les indices pour toutes les anches mesurées de la série
    Ncibles = len(cibles)
    COLORS = plt.cm.cool(np.linspace(0, 1, Ncibles))

    Xcibles = np.full(Ncibles, np.nan)
    Ycibles = np.full(Ncibles, np.nan)


    for i in range(2): 
        plt.figure()
        plt.clf()
        plt.imshow(I, cmap='gray', extent=[0, W, H, 0])
        plt.axis('equal')
        plt.xlim(0, W)
        plt.ylim(0, H)
        plt.axhline(H, color='r')
        
        for j in range(1, i):
            # Tracer le carré
            plt.plot([Xcibles[j] - a, Xcibles[j] + a, Xcibles[j] + a, Xcibles[j] - a, Xcibles[j] - a],
                    [Ycibles[j] - a, Ycibles[j] - a, Ycibles[j] + a, Ycibles[j] + a, Ycibles[j] - a],
                    color=COLORS[j])

            # Tracer le marqueur '+'
            plt.plot(Xcibles[j], Ycibles[j], '+', color=COLORS[j])

        if i > Ncibles:
            break
        
        plt.title('Cliquer sur 1 = Anche Gauche, 2 = Anche Droite -> {} ({})'.format(i, cibles[i]))
        xclic, yclic = plt.ginput(1)[0]
        c1 = np.array([round(xclic), round(yclic)])  # prendre des entiers
        plt.title('')
        
        Xcibles[i] = c1[0]  # pixel x des cibles
        Ycibles[i] = c1[1]  # pixel y des cibles
        # print(f'Ycibles {Ycibles}')


    Xv = np.arange(Xcibles[0], Xcibles[1], dtype=int)
    # print(f'Xv {Xv}')
    # print(np.shape(Xv))

    # # Detection du profil pour toutes les forces d'une anche
    for F in range(2):
        I = cv2.imread(dossier + nom_anche + str(R+1) + '_' + str(F) + '.bmp', cv2.IMREAD_GRAYSCALE)
        I = np.flipud(I)
        I = I[150:hmax, :]
        # print('debug 1')

        # Yv_m = np.zeros_like(Xv)
        # Yv_r = np.zeros_like(Xv)
        Yv_m = []
        Yv_r = []

# PROBLEME POUR GENERALISER A UNE TRANCHE ENTIERE
        Iv = np.empty([len(Xv)+1, 1])
        for xv in Xv:
            Iv = I[:, xv]  # vecteur qui contient une tranche de l'image largeur 1 (x) et hauteur de l'image (y)
            Iv = Iv.astype(float)
            # print('debug 2')
            # print(f'Xv {Xv}')
            # print(f'Iv {Iv}')
            # print(type(Xv))
            # print(np.shape(Iv))

            # Iv_2 = np.diff(Iv)  # dérivée
            Iv_2_float = np.gradient(Iv)  # Dérivée centrée (flottante)
            Iv_2 = np.round(Iv_2_float).astype(int)  # Arrondir et convertir en entiers
            # print(f'IV_2 {Iv_2}')
            # print(type(Iv_2))
            # print(np.shape(Iv_2))


            smin = -13
            smax = 10
            pos1 = np.where(Iv_2 < smin)[0][-1] if any(Iv_2 < smin) else None
            pos2 = np.where(Iv_2 > smax)[0][-1] if any(Iv_2 > smax) else None
        
            # print(f'pos1 {pos1}')
            # print(f'pos2 {pos2}')
            # print(f'Y : {Y}, {len(Y)}')
            # print(f'Yv_m : {Yv_m}, {len(Yv_m)}')
            
            if pos2 is not None:
                # Yv_m[i] = Y[pos2]
                Yv_m.append(Y[pos2])
            else:
                # Yv_m[i] = np.nan
                Yv_m.append(np.nan)
                print('error!!!')  # Peu probable parce que détectera toujours la frontière lèvre/anche

            if pos1 is not None:
                # Yv_r[i] = Y[pos1]
                Yv_r.append(Y[pos1])
            else:
                # Yv_r[i] = Yv_m[i]  # Lorsque le canal d'anche est fermé, il n'y aura pas de Yv_r. On lui donne la même valeur que Yv_m comme ça la différence sera de 0.
                Yv_r.append(Yv_m)
                
                # print(f'Yv_r {Yv_r}')
                # print(f'Yv_m {Yv_m}')
                print(type(Yv_r))
                print(np.shape(Yv_r))
                print(type(Xv))
                print(np.shape(Xv))

        plt.plot(Xv, Yv_r, '.-', linewidth=0.1)
        plt.plot(Xv, Yv_m, 'm.-', linewidth=0.1)
        plt.pause(1)

    #     # Calcule de la surface du canal d'anche
        pix_canal = Yv_m - Yv_r
    #     # print(f'pix_canal {pix_canal}')

        dist_px = 1064  # mesuré sur l'image MIRE 05-03-42 (8 carreaux)
        dist_mm = 12  # mesuré à la règle sur la mire (cf mon cahier de labo, 2 carreaux = 3mm)
        S = dist_mm / dist_px  # "sensibilité" (en mm/px)

        nb_pix_canal = np.sum(pix_canal)  # je compte tous les pixels qu'il y a dans le canal
        surf_pix = S ** 2  # surface d'un pixel
        dim_canal = nb_pix_canal * surf_pix  # surface en mm² du canal

        # print('Surface canal estime =', int(dim_canal), 'mm²')

        surf_c[F] = int(dim_canal)
        # print(f'Yv_r {Yv_r}')
        # def_r[F] = int(Yv_r)
        # def_m[F] = Yv_m
        
    # # savename = nom_anche + str(R) + '.mat'
    # savename = nom_anche + str(R) + '.txt'
    # np.savetxt(dossier + savename, 'def_r', 'def_m', 'surf_c')
    
# plt.show()

# # Appel de la fonction
# dossier = "MAD Statique/mes_2024.03.22/"
# nom_anche = 'A'
# detect_contour(dossier, nom_anche)


(1080, 1920)
<class 'list'>
(1, 1)
<class 'numpy.ndarray'>
(1099,)
<class 'list'>
(2, 2)
<class 'numpy.ndarray'>
(1099,)
<class 'list'>
(3, 3)
<class 'numpy.ndarray'>
(1099,)
<class 'list'>
(4, 4)
<class 'numpy.ndarray'>
(1099,)
<class 'list'>
(5, 5)
<class 'numpy.ndarray'>
(1099,)
<class 'list'>
(6, 6)
<class 'numpy.ndarray'>
(1099,)
<class 'list'>
(7, 7)
<class 'numpy.ndarray'>
(1099,)
<class 'list'>
(8, 8)
<class 'numpy.ndarray'>
(1099,)
<class 'list'>
(9, 9)
<class 'numpy.ndarray'>
(1099,)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1099,) + inhomogeneous part.

### Traitement des données extraites

In [ ]:
# tracer la surface du cannal d'anche

plt.close('all')

plt.figure(figsize=(8, 6))
dossier = 'MAD Statique/mes_2024.03.22'
pentes = []
# surf_c = []
# F =[]
for i in range (3): 
    surf_c = np.loadtxt(dossier + f'/A{i+1}.txt')
    F = np.loadtxt(dossier + f'/force_A{i+1}.txt')
    
 
    # extractions des paramètres de la courbe paramétrique
    coefficients = np.polyfit(F, surf_c, 3)
    pente = coefficients[0]
    pente = round(pente, 2)
    ordonnee_origine = coefficients[1]
    ordonnee_origine = round(ordonnee_origine, 2)
    pentes.append(pente)
    
    plt.plot(surf_c, np.polyval(coefficients, surf_c), linestyle='--', label=f'courbe param')


    # Plot
    plt.plot(F, surf_c, '*-', linewidth=1.8, label=f'Force {i+1}')
    plt.xlabel('Force')
    plt.ylabel('Surface (mm²)')
    plt.title('Surface du canal d\'anche en fonction de la force \n d\'appui de la lèvre articielle')
    plt.legend()
    plt.grid(True)
    plt.axis([-30, 350, -2, 13])
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()


ValueError: the number of columns changed from 1069 to 1 at row 61; use `usecols` to select a subset and avoid this error

In [ ]:
# tracer la surface du cannal d'anche

plt.close('all')

plt.figure(figsize=(8, 6))
dossier = 'MAD Statique/mes_2024.03.22'
pentes = []
# surf_c = []
# F =[]
for i in range (3): 
    surf_c = np.loadtxt(dossier + f'/A{i+1}.txt')
    F = np.loadtxt(dossier + f'/force_A{i+1}.txt')
    
 
    # PlotF
    plt.plot(F, surf_c, '*-', linewidth=1.8, label=f'Force {i+1}')
    identification_modele(F, surf_c)
    plt.xlabel('Force')
    plt.ylabel('Surface (mm²)')
    plt.title('Surface du canal d\'anche en fonction de la force \n d\'appui de la lèvre articielle')
    plt.legend()
    plt.grid(True)
    plt.axis([-30, 350, -2, 13])
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()


ValueError: the number of columns changed from 1069 to 1 at row 61; use `usecols` to select a subset and avoid this error